In [1]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# env = gym.make('FrozenLake-v0')

In [ ]:
def run(mode='noise'):
    Q = np.zeros((env.observation_space.n, env.action_space.n))
    dis = .99
    num_episodes = 2000 
    lr = .85
    r_list = []
    for i in range(num_episodes):
        e = 1. / ((i // 100) + 1)
        state = env.reset()
        r_all = 0
        done = False

        while not done:
            if mode == 'noise':
                action = np.argmax(Q[state] + np.random.randn(1, env.action_space.n) / (i + 1))
            else:
                action = env.action_space.sample() if np.random.rand(1) < e else np.argmax(Q[state])       
            new_state, reward, done, _ = env.step(action)
            Q[state, action] = (1 - lr) * Q[state, action] + \
                lr * (reward + dis * np.max(Q[new_state]))
            r_all += reward
            state = new_state
        r_list.append(r_all)
    print(f'Success rate: {(sum(r_list) / num_episodes):.3f}')
    print('Final Q-Table Values')
    print(Q)
    plt.plot(range(len(r_list)), r_list, color='blue')
    plt.show()

In [ ]:
run(mode='e-greedy')

In [ ]:
def one_hot(x):
    return np.identity(16)[x: x + 1]

In [ ]:
input_size = env.observation_space.n
output_size = env.action_space.n
lr = 0.1
X = tf.placeholder(shape=[1, input_size], dtype=tf.float32)
W = tf.Variable(tf.random_uniform([input_size, output_size], 0, 0.01))
Q_pred = tf.matmul(X, W)
Y = tf.placeholder(shape=[1, output_size], dtype=tf.float32)
loss = tf.reduce_sum(tf.square(Y - Q_pred))
train = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(loss)

dis = .99
num_episodes = 2000
r_list = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_episodes):
        s = env.reset()
        e = 1. / ((i / 50) + 10)
        r_all = 0
        done = False
        local_loss = []

        while not done:
            Q_s = sess.run(Q_pred, feed_dict={X: one_hot(s)})
            if np.random.rand(1) < e:
                a = env.action_space.sample()
            else:
                a = np.argmax(Q_s)

            s1, reward, done, _ = env.step(a)
            if done:
                Q_s[0, a] = reward
            else:
                Q_s1 = sess.run(Q_pred, feed_dict={X: one_hot(s1)})
                Q_s[0, a] = reward + dis * np.max(Q_s1)
            sess.run(train, feed_dict={X: one_hot(s), Y: Q_s})
            r_all += reward
            s = s1
        r_list.append(r_all)
    print(f'Success rate: {(sum(r_list) / num_episodes):.3f}')
    print('Final Q-Table Values')
    plt.plot(range(len(r_list)), r_list, color='blue')
    plt.show()

In [2]:
env = gym.make('CartPole-v0')
input_size = env.observation_space.shape[0]
output_size = env.action_space.n
lr = 0.1
X = tf.placeholder(shape=[None, input_size], name="input_x", dtype=tf.float32)
W1 = tf.get_variable("W1", shape=[input_size, output_size], initializer=tf.contrib.layers.xavier_initializer())
Q_pred = tf.matmul(X, W1)

W0728 00:33:24.431920 140735572104064 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
Y = tf.placeholder(shape=[None, output_size], dtype=tf.float32)
loss = tf.reduce_sum(tf.square(Y - Q_pred))
train = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

In [7]:
dis = .99
num_episodes = 2000
r_list = []

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(num_episodes):
        s = env.reset()
        e = 1. / ((i / 50) + 10)
        step_count = 0
        done = False
        local_loss = []

        while not done:
            step_count += 1
            x = s.reshape(1, input_size)
            Q_s = sess.run(Q_pred, feed_dict={X: x})
            if np.random.rand(1) < e:
                a = env.action_space.sample()
            else:
                a = np.argmax(Q_s)

            s1, reward, done, _ = env.step(a)
            if done:
                Q_s[0, a] = -100
            else:
                x1 = s1.reshape(1, input_size)
                Q_s1 = sess.run(Q_pred, feed_dict={X: x1})
                Q_s[0, a] = reward + dis * np.max(Q_s1)
            sess.run(train, feed_dict={X: x, Y: Q_s})
            s = s1
        r_list.append(step_count)
        if len(r_list) > 10 and np.mean(r_list[-10:]) > 500:
            break
    observation = env.reset()
    reward_sum = 0
    while True:
        x = observation.reshape(1, input_size)
        Q_s = sess.run(Q_pred, feed_dict={X: x})
        observation, reward, done, _ = env.step(a)
        reward_sum += reward
        if done:
            print(f'Total Score: {reward_sum}')
            break

Total Score: 11.0
